# Setup

In [22]:
from datetime import date

import pandas as pd
import boto3
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib

from lib.helpers import subset_df_by_id, resource_formatter, project_formatter, format_current_trends
from lib import TrendFinder as tf
from lib import overview_traces as ot
from lib import demo
from lib import geo as g
from lib import plot_formatters as pf 

In [5]:
# Style for TrendFinder email
style = """
<style>
body {
    font-family: "Georgia", sans-serif;
}

    table {
    border-collapse: collapse;
}
th, td {
    padding: 3px;
    text-align: left;
}
table, th, td {
    border: 1px solid black;
    font-family: Futura;
    font-size: 12px;
    font-weight: normal;
}

th {
    background-color: #065331;
    color: white;
    
}
</style>
"""

In [ ]:
# By default, date is today
DATE = date.today().strftime("%Y-%m-%d")

In [25]:
pd.options.mode.chained_assignment = None

## Configuration

In [9]:
# AWS initialization
bucket = 'donorschoose-trends' # s3 bucket name
client = boto3.client("s3")
s3 = boto3.resource('s3')

Credentials need to be set using awscli (see Directions).

## Pipeline wrapper functions

In [26]:
# Overview (does subset_df at a time)
def build_overview(word, subset_df):
    metro_plot = ot.plot_by_metro(subset_df, word, plot=False)
    income_plot = ot.plot_by_income(subset_df, word, plot=False)
    subject_plot = ot.plot_by_subject(subset_df, word, plot=False)
    grade_plot = ot.plot_by_grade(subset_df, word, plot=False)
    
    pf.output_plot_data(word, metro_plot, 'plot_by_metro', DATE, bucket, client)
    pf.output_plot_data(word, income_plot, 'plot_by_income', DATE, bucket, client)
    pf.output_plot_data(word, subject_plot, 'plot_by_subject', DATE, bucket, client)
    pf.output_plot_data(word, grade_plot, 'plot_by_grade', DATE, bucket, client)

    metro_percent = ot.percent_by_metro(subset_df, word, plot=False)
    income_percent = ot.percent_by_income(subset_df, word, plot=False)
    subject_percent = ot.percent_by_subject(subset_df, word, plot=False)
    grade_percent = ot.percent_by_grade(subset_df, word, plot=False)

    pf.output_plot_data(word, metro_percent, 'percent_by_metro', DATE, bucket, client)
    pf.output_plot_data(word, income_percent, 'percent_by_income', DATE, bucket, client)
    pf.output_plot_data(word, subject_percent, 'percent_by_subject', DATE, bucket, client)
    pf.output_plot_data(word, grade_percent, 'percent_by_grade', DATE, bucket, client)

In [27]:
# Demographics (does all at once, depends on projects and keyword_ids_dict)
def build_demo():
    cor = demo.Correlator(projects)
    cor.find_trends(keywords_dict = keyword_ids_dict)

    # Default time_interval = 1M
    cor.get_categorical_trends(trend_keywords, prop=False, thres=-1)
    cor.stationarize()
    cor.stationarity_test_all() # what passes this test is in .passed_trends
    cor.compare_corrs()

    # Get trends that did not pass
    demo_not_passed = [word for word in trend_keywords if word not in cor.passed_trends]

    features = [col for col in cor.df.columns if 'Bin' in col]

    for word in cor.passed_trends:
        top_corrs = cor.top_corrs(word)
        pf.output_table_data(word, top_corrs, "top_corrs", DATE, bucket, client, index=True)
        trend_features_out = demo.plot_trend_features(cor.grouped, trend=word, passed_features = cor.passed_features, date_cutoff=trend_finder.current_start, plot=False)
        pf.output_plot_data(word, trend_features_out, 'plot_trend_features', DATE, bucket, client)
        
    for word in trend_keywords:
        # Ratios
        diffs = demo.compare_ratios(cor.df, cor.grouped, trend=word, features=features)
        diffs_out = demo.plot_diffs(diffs, feat_type=['Poverty', 'Metro', 'Grade', 'Various'], plot=False)
        pf.output_plot_data(word, diffs_out, 'plot_diffs', DATE, bucket, client)

        # Google Trends
        google_trends = demo.ggl_trends(cor.grouped, word)
        ggl_trends_out = demo.plot_ggl_trends(google_trends, word, plot=False)
        pf.output_plot_data(word, ggl_trends_out, 'plot_ggl_trends', DATE, bucket, client)

In [28]:
# Geo (does subset_df at a time)
def build_geo(word, subset_df):
    geo = g.GeoMeta(subset_df)
    
    geo.get_all_splits()
    trendiest = geo.find_trendiest(as_df=True)
    pf.output_table_data(word, trendiest, "geo_splits", DATE, bucket, client, index=True)
    
    plot_splits_out = geo.plot_splits(word, plot=False)
    pf.output_plot_data(word, plot_splits_out, 'plot_splits', DATE, bucket, client)
    
    windows = [geo.ONE_MONTH, geo.THREE_MONTHS, geo.SIX_MONTHS, geo.ONE_YEAR]
    for window in windows:
        plot_rolling_out = geo.plot_rolling_splits(word, window=window, plot=False)
        pf.output_plot_data(word, plot_rolling_out, 'plot_rolling_splits_{}'.format(window), DATE, bucket, client)
    
    plot_cumulative_out = geo.plot_cumulative_splits(word, plot=False)
    pf.output_plot_data(word, plot_cumulative_out, 'plot_cumulative_splits', DATE, bucket, client)

# Detect trends

## Read in resources

In [9]:
#########################################################################################
# Expecting a .csv with Project ID, Project Posted Date, and Item Cleaned Resource Name #
#########################################################################################

# Read in resources (takes a couple of minutes)
resources = resource_formatter("/shared-files/csv/new_resources_only.csv")

Reading in resource data...
Consolidating resources per project for 1078130 projects ...
Resources read and formatted!
CPU times: user 2min 49s, sys: 5.62 s, total: 2min 55s
Wall time: 2min 55s


In [30]:
# Create TrendFinder object
trend_finder = tf.TrendFinder(resources)
current_trends = trend_finder.find_current_trends()

# Save keywords to list
trend_keywords = list(current_trends["word"])

Cleaning...
Performing date operations...
Reducing strings to list of unique words...
Cleaning done!
Looking at projects from 2018-03-14 to 2018-03-27.
There are 9207 projects in the current time frame.

Building frequency dictionary...
Frequency dictionary built!
Total words: 1238

Creating groups for iteration...
Building history...
Looking 576 days back to test against current time frame.

61 keywords deviate more than 2 SDs above their normal mean.

Time elapsed: 0.18943584362665813 minutes.
CPU times: user 33.2 s, sys: 1.11 s, total: 34.4 s
Wall time: 34.3 s


In [31]:
current_trends_table = format_current_trends(current_trends)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.2 ms


In [32]:
# Transition from resources to file structure
# del resources # For memory

## Email trends

In [ ]:
email_trends = current_trends_table.to_html(index=False)

In [8]:
body = """
<p>Welcome to TrendFinder! Below are the trending keywords for the last two weeks.<br>

To view the dashboard, <a href='https://trendfinder.elasticbeanstalk.com'>click here!</a>
</p>
<br>
"""

In [ ]:
# Create HTML of email
table_html = "<html>" + style + body + table + "</html>"

In [ ]:
# Put recipients here
recipients = ['DC_USER@gmail.com']

In [ ]:
# Create message information
msg = MIMEMultipart('alternative')
msg['Subject'] = "TrendFinder Dashboard"
msg['From'] = 'DC_USER@gmail.com'
msg['To'] = ','.join(recipients)

In [ ]:
# Add message contents
email_table = MIMEText(table_html, 'html')
msg.attach(email_table)

In [ ]:
# Initiate server and send email
server = smtplib.SMTP_SSL('smtp.gmail.com', 465) # SMTP port for sending email
server.login("DC_USER", "PASSWORD") # Fill this in
server.sendmail(
  "DC_USER@gmail.com", # Put in origin email
  recipients,
  msg.as_string())
server.quit()

## Save trends

In [33]:
pf.output_table_data(trend=None, df=current_trends_table, table_name=None, prefix=DATE, bucket=bucket, s3_client=client, index=False)

## Build keyword-IDs dictionary

In [34]:
# Get dictionary of Project IDs per keyword
keyword_ids_dict = {}

for word in trend_keywords:
    # DataFrame subset for low income
    keyword_ids_dict[word] = trend_finder.subset_resources_by_query(word)["Project ID"].tolist()

CPU times: user 23.3 s, sys: 0 ns, total: 23.3 s
Wall time: 23.3 s


# Investigate trends

In [35]:
#########################################
# Expecting a .csv with project_columns #
#########################################

# Read in project info
projects = project_formatter("/shared-files/csv/new_project_info.csv")

Reading in project data...
Projects read and formatted!
CPU times: user 5.71 s, sys: 160 ms, total: 5.87 s
Wall time: 5.86 s


## Co-occurrences

In [36]:
# Find co-occurrences
co_occurrences_dict = {}

for word in trend_keywords:
    co_occurrences_dict[word] = pd.DataFrame(trend_finder.find_co_occurrences(word), columns = ["Word", "Count"])
    pf.output_table_data(word, co_occurrences_dict[word], "co_occurrences", DATE, bucket, client, index=True)

CPU times: user 976 ms, sys: 0 ns, total: 976 ms
Wall time: 3.58 s


## plot_xox

In [37]:
# Overall trend history
for word in trend_keywords:
    plot_xox_out = trend_finder.plot_xox(word, plot = False)
    pf.output_plot_data(word, plot_xox_out, 'plot_xox', DATE, bucket, client)

CPU times: user 25 s, sys: 0 ns, total: 25 s
Wall time: 28.2 s


## Demo

In [38]:
build_demo()

Computing trends
Done!
insects
insect
flex
bugs
bug
butterfly
	Failed!
touch
bulk
butterflies
caterpillars
live
lc405bu
	Failed!
cycle
lore
ladybug
	Failed!
habitat
lc405gr
	Failed!
id
wiggle
caterpillar
spiders
real
metallic
foldable
	Failed!
explore
	Failed!
stickers
growing
bee
fits
	Failed!
flower
7mm
	Failed!
use
	Failed!
battery
	Failed!
supplies
	Failed!
easter
ultimate
basket
jr
	Failed!
master
	Failed!
wholesale
lc511bu
k04us
foil
	Failed!
toxic
frito
touchscreen
lay
required
	Failed!
tank
ips
	Failed!
universe
toddlers
	Failed!
personal
alternative


/home/pbadami/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1036: RuntimeWarning:

invalid value encountered in true_divide



	Failed!
stream
256gb
ez
handheld
ideal
track
violet
	Failed!
Bin_Project Grade Level(Grades 6-8)
Bin_Project Grade Level(Grades PreK-2)
Bin_Project Grade Level(Grades 3-5)
Bin_Project Grade Level(Grades 9-12)
Bin_School Metro Area(suburban)
Bin_School Metro Area(urban)
Bin_School Metro Area(rural)
Bin_School Metro Area(town)
Bin_Project Subject(Music)
Bin_Project Subject(Special Needs)
	Failed!
Bin_Project Subject(Literature & Writing)
Bin_Project Subject(Literacy)
Bin_Project Subject(Health & Wellness)
Bin_Project Subject(Mathematics)
	Failed!
Bin_Project Subject(Environmental Science)
	Failed!
Bin_Project Subject(Financial Literacy)
Bin_Project Subject(Applied Sciences)
Bin_Project Subject(Visual Arts)
Bin_Project Subject(Character Education)
	Failed!
Bin_Project Subject(Other)
	Failed!
Bin_Project Subject(Early Development)
	Failed!
Bin_Project Subject(Warmth, Care & Hunger)
Bin_Project Subject(Health & Life Science)
Bin_Project Subject(Performing Arts)
	Failed!
Bin_Project Subject

                                                     insect
Bin_Project Subject(Health & Life Science)         0.674185
Bin_School Metro Area(suburban)                    0.670582
Bin_Project Subject(Music)                         0.662787
Bin_Project Grade Level(Grades 6-8)                0.657803
Bin_Project Grade Level(Grades 9-12)               0.638119
Bin_Project Subject Category(Music & The Arts)     0.632310
Bin_Magnet(Yes)                                    0.619453
Bin_Project Grade Level(Grades 3-5)                0.608347
Bin_School Metro Area(urban)                       0.601811
Bin_Poverty(low income)                            0.598949
Bin_Project Subject(History & Geography)           0.597916
Bin_Project Subject Category(History & Civics)     0.591004
Bin_Charter(Yes)                                   0.585366
Bin_Project Subject(Literature & Writing)          0.580496
Bin_Project Subject(Visual Arts)                   0.578361
Bin_Project Subject(Applied Sciences)   

                                                   caterpillars
Bin_Project Subject(Health & Life Science)             0.654577
Bin_School Metro Area(suburban)                        0.635939
Bin_Project Subject(Music)                             0.622544
Bin_Project Grade Level(Grades 6-8)                    0.622517
Bin_Project Subject Category(Music & The Arts)         0.588650
Bin_Magnet(Yes)                                        0.587582
Bin_Project Subject(History & Geography)               0.578666
Bin_Project Subject Category(History & Civics)         0.573742
Bin_Year_Round(Yes)                                    0.573153
Bin_Project Grade Level(Grades 3-5)                    0.571008
Bin_Project Grade Level(Grades 9-12)                   0.565882
Bin_School Metro Area(urban)                           0.562720
Bin_Poverty(low income)                                0.561695
Bin_Project Grade Level(Grades PreK-2)                 0.559602
Bin_Project Subject(Applied Sciences)   

                                                       real
Bin_Project Subject Category(History & Civics)     0.861875
Bin_Project Grade Level(Grades 6-8)                0.817068
Bin_Project Subject Category(Literacy & Language)  0.799121
Bin_Project Subject(Literacy)                      0.797840
Bin_Magnet(Yes)                                    0.793571
Bin_Project Subject(History & Geography)           0.791411
Bin_School Metro Area(suburban)                    0.790284
Bin_Project Grade Level(Grades 3-5)                0.786399
Bin_Project Subject(Foreign Languages)             0.773902
Bin_Charter(Yes)                                   0.772524
Bin_Poverty(low income)                            0.770347
Bin_School Metro Area(urban)                       0.765822
Bin_Project Subject(Music)                         0.763203
Bin_School Metro Area(town)                        0.761041
Bin_Project Grade Level(Grades 9-12)               0.756943
Bin_Project Subject(Health & Life Scienc

Empty DataFrame
Columns: [easter]
Index: []
                                                   ultimate
Bin_Project Grade Level(Grades 3-5)                0.769986
Bin_Project Subject(Literacy)                      0.742662
Bin_Project Subject Category(Literacy & Language)  0.735148
Bin_Poverty(low income)                            0.734892
Bin_School Metro Area(urban)                       0.733141
Bin_Project Grade Level(Grades PreK-2)             0.731988
Bin_School Metro Area(suburban)                    0.729341
Bin_Project Grade Level(Grades 6-8)                0.718043
Bin_Project Subject(History & Geography)           0.714456
Bin_Project Subject(Literature & Writing)          0.711922
Bin_Project Subject(Music)                         0.710089
Bin_Charter(Yes)                                   0.710086
Bin_Magnet(Yes)                                    0.709958
Bin_School Metro Area(rural)                       0.706842
Bin_School Metro Area(town)                        0.704

                                                       tank
Bin_Project Grade Level(Grades 6-8)                0.732331
Bin_Charter(Yes)                                   0.724202
Bin_Project Grade Level(Grades 3-5)                0.719035
Bin_School Metro Area(suburban)                    0.711605
Bin_Poverty(low income)                            0.706822
Bin_School Metro Area(urban)                       0.705840
Bin_Magnet(Yes)                                    0.691961
Bin_Project Subject(Literature & Writing)          0.682609
Bin_Project Grade Level(Grades PreK-2)             0.682225
Bin_Project Grade Level(Grades 9-12)               0.679962
Bin_Project Subject(Music)                         0.676503
Bin_School Metro Area(town)                        0.663777
Bin_Project Subject(Health & Life Science)         0.657492
Bin_School Metro Area(rural)                       0.656945
Bin_Project Subject Category(Music & The Arts)     0.653742
Bin_Project Subject(History & Geography)

                                                   handheld
Bin_Project Grade Level(Grades 9-12)               0.761059
Bin_Project Grade Level(Grades 6-8)                0.748265
Bin_School Metro Area(suburban)                    0.747011
Bin_Project Subject(Health & Life Science)         0.733598
Bin_Project Grade Level(Grades 3-5)                0.716738
Bin_Project Subject(Music)                         0.712620
Bin_Magnet(Yes)                                    0.693681
Bin_Project Subject(Literature & Writing)          0.689197
Bin_Poverty(low income)                            0.688429
Bin_Project Subject Category(Music & The Arts)     0.687276
Bin_School Metro Area(urban)                       0.681768
Bin_Project Subject(History & Geography)           0.680886
Bin_Project Subject(Foreign Languages)             0.676248
Bin_Charter(Yes)                                   0.666951
Bin_Project Grade Level(Grades PreK-2)             0.661998
Bin_School Metro Area(town)             

## Overview/Geo

In [39]:
for word in trend_keywords:
    # Get subset of projects for word
    subset_df = subset_df_by_id(projects, keyword_ids_dict[word])
    # Overview
    build_overview(word, subset_df)
    # Geo
    build_geo(word, subset_df)

CPU times: user 1min 36s, sys: 256 ms, total: 1min 36s
Wall time: 2min 39s


Done!